In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e23/sample_submission.csv
/kaggle/input/playground-series-s3e23/train.csv
/kaggle/input/playground-series-s3e23/test.csv


In [27]:
train = pd.read_csv('/kaggle/input/playground-series-s3e23/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s3e23/test.csv', index_col='id')

print(train.shape)
print(test.shape)

(101763, 22)
(67842, 21)


In [28]:
features = [f for f in train.columns.tolist() if f != 'defects']
print(features)

['loc', 'v(g)', 'ev(g)', 'iv(g)', 'n', 'v', 'l', 'd', 'i', 'e', 'b', 't', 'lOCode', 'lOComment', 'lOBlank', 'locCodeAndComment', 'uniq_Op', 'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount']


In [29]:
test['l'] = 1.0 - test['l']
train['l'] = 1.0 - train['l']

In [30]:
X = train[features]
y = train['defects']
test = test[features].copy()

In [31]:
from lightgbm import LGBMClassifier

model_LGBM = LGBMClassifier(n_estimators=20000,
                            learning_rate=0.07,
                            objective='binary',
                            boosting_type='gbdt',
                            subsample=1.0,
                            num_leaves=23,
                            max_bin=1023,n_jobs=-1,
                            reg_alpha=0.65,
                            reg_lambda=3.1,
                            colsample_bytree=0.568,
                            min_child_samples=864,
                            random_state=1920)

In [32]:
from sklearn.ensemble import HistGradientBoostingClassifier

model_HGB = HistGradientBoostingClassifier(max_iter=250,
                                           validation_fraction=None,
                                           learning_rate=0.007,
                                           max_depth=10,
                                           min_samples_leaf=24,
                                           max_leaf_nodes=60,
                                           random_state=1920,
                                           verbose=0)

In [36]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import ExtraTreesClassifier

model_ET = make_pipeline(ColumnTransformer([('drop', 'drop', 
                                              ['iv(g)', 't', 'b', 'n', 'lOCode', 'v',
                                               'branchCount', 'e', 'i', 'lOComment'])],
                                            remainder='passthrough'),
                          FunctionTransformer(np.log1p),
                          ExtraTreesClassifier(n_estimators=100,
                                               min_samples_leaf=100,
                                               max_features=1.0,
                                               random_state=1))


In [40]:
import random

def ensemble(model, model0):
    counter = 0
    auc_mean = 0
    preds = np.zeros(len(test))
    rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1920)

    for fold, (train_idx, valid_idx) in enumerate(rkf.split(X)):  
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_valid, y_valid = X.iloc[valid_idx], y.iloc[valid_idx]  

        print(f'\n:::::::::::::::::: Fold ~ {fold+1} :::::::::::::::::::')

        N = random.randrange(4) 
        if (N == 0):
            print('HGBClassifier >>')
            model0.fit(X_train, y_train)
            oof = model0.predict_proba(X_valid)[:, -1]

        if (N != 0):
            print('LGBMClassifier >>\n')
            model.fit(X_train, y_train,             
                      eval_set=[(X_valid, y_valid)], 
                      eval_metric='auc', 
                      early_stopping_rounds=250,            
                      verbose=100)
            oof = model.predict_proba(X_valid)[:, -1]

        auc = roc_auc_score(y_valid, oof)
        if (auc <= 0.79): 
            print('\nAUC Score:', auc, ' # was ignored.')

        if (auc > 0.79): 
            counter += 1
            print('\nAUC Score:', auc, ' # it is ok.')
            auc_mean += roc_auc_score(y_valid, oof) 
            preds += model.predict_proba(test)[:, -1] 

    auc_mean = auc_mean / counter      
    preds = preds / counter 

    print('\n', '='* 40)
    print(' .'* 20)
    print(' AUC Score (mean):', auc_mean)
    print(' .'* 20)
    print('='* 40, '\n')

    print('Total number of folds:', rkf.get_n_splits(X, y))
    print('Number of valid folds:', counter)
    
    
ensemble(model_LGBM, model_HGB)


:::::::::::::::::: Fold ~ 1 :::::::::::::::::::
LGBMClassifier >>



/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's auc: 0.794126	valid_0's binary_logloss: 0.429823
[200]	valid_0's auc: 0.793999	valid_0's binary_logloss: 0.429956
[300]	valid_0's auc: 0.793376	valid_0's binary_logloss: 0.430526

AUC Score: 0.7944049387616356  # it is ok.

:::::::::::::::::: Fold ~ 2 :::::::::::::::::::
HGBClassifier >>

AUC Score: 0.7831124066891859  # was ignored.

:::::::::::::::::: Fold ~ 3 :::::::::::::::::::
LGBMClassifier >>



/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's auc: 0.798421	valid_0's binary_logloss: 0.427961
[200]	valid_0's auc: 0.798253	valid_0's binary_logloss: 0.428416
[300]	valid_0's auc: 0.797324	valid_0's binary_logloss: 0.429332

AUC Score: 0.7985195777300689  # it is ok.

:::::::::::::::::: Fold ~ 4 :::::::::::::::::::
LGBMClassifier >>



...

[100]	valid_0's auc: 0.793531	valid_0's binary_logloss: 0.429537
[200]	valid_0's auc: 0.793734	valid_0's binary_logloss: 0.429626
[300]	valid_0's auc: 0.793399	valid_0's binary_logloss: 0.429979

AUC Score: 0.7938760769857142  # it is ok.

:::::::::::::::::: Fold ~ 49 :::::::::::::::::::
LGBMClassifier >>



/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's auc: 0.792507	valid_0's binary_logloss: 0.424602
[200]	valid_0's auc: 0.791895	valid_0's binary_logloss: 0.425169
[300]	valid_0's auc: 0.790962	valid_0's binary_logloss: 0.425966

AUC Score: 0.7925425452192368  # it is ok.

:::::::::::::::::: Fold ~ 50 :::::::::::::::::::
LGBMClassifier >>



/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's auc: 0.794515	valid_0's binary_logloss: 0.428843
[200]	valid_0's auc: 0.794232	valid_0's binary_logloss: 0.429041
[300]	valid_0's auc: 0.793856	valid_0's binary_logloss: 0.429305

AUC Score: 0.7946002121657749  # it is ok.

 . . . . . . . . . . . . . . . . . . . .
 AUC Score (mean): 0.7938140339739052
 . . . . . . . . . . . . . . . . . . . .

Total number of folds: 50
Number of valid folds: 35


In [41]:
import random

def ensemble_new(model0, model1, model):
    counter = 0
    auc_mean = 0
    preds = np.zeros(len(test))
    rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1920)

    for fold, (train_idx, valid_idx) in enumerate(rkf.split(X)):  
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_valid, y_valid = X.iloc[valid_idx], y.iloc[valid_idx]  

        print(f'\n:::::::::::::::::: Fold ~ {fold+1} :::::::::::::::::::')

        N = random.randrange(4) 
        if (N == 0):
            print('HGBClassifier >>')
            model0.fit(X_train, y_train)
            oof = model0.predict_proba(X_valid)[:, -1]

        if (N == 1):
            print('ExtraTreesClassfier >>')
            model1.fit(X_train, y_train)
            oof = model1.predict_proba(X_valid)[:, -1]
            
        if (N != 0):
            print('LGBMClassifier >>\n')
            model.fit(X_train, y_train,             
                      eval_set=[(X_valid, y_valid)], 
                      eval_metric='auc', 
                      early_stopping_rounds=250,            
                      verbose=100)
            oof = model.predict_proba(X_valid)[:, -1]

        auc = roc_auc_score(y_valid, oof)
        if (auc <= 0.79): 
            print('\nAUC Score:', auc, ' # was ignored.')

        if (auc > 0.79): 
            counter += 1
            print('\nAUC Score:', auc, ' # it is ok.')
            auc_mean += roc_auc_score(y_valid, oof) 
            preds += model.predict_proba(test)[:, -1] 

    auc_mean = auc_mean / counter      
    preds = preds / counter 

    print('\n', '='* 40)
    print(' .'* 20)
    print(' AUC Score (mean):', auc_mean)
    print(' .'* 20)
    print('='* 40, '\n')

    print('Total number of folds:', rkf.get_n_splits(X, y))
    print('Number of valid folds:', counter)
    
    
ensemble_new(model_HGB, model_ET, model_LGBM)


:::::::::::::::::: Fold ~ 1 :::::::::::::::::::
ExtraTreesClassfier >>
LGBMClassifier >>



/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's auc: 0.794126	valid_0's binary_logloss: 0.429823
[200]	valid_0's auc: 0.793999	valid_0's binary_logloss: 0.429956
[300]	valid_0's auc: 0.793376	valid_0's binary_logloss: 0.430526

AUC Score: 0.7944049387616356  # it is ok.

:::::::::::::::::: Fold ~ 2 :::::::::::::::::::
LGBMClassifier >>



/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's auc: 0.784187	valid_0's binary_logloss: 0.431646
[200]	valid_0's auc: 0.784466	valid_0's binary_logloss: 0.431662
[300]	valid_0's auc: 0.784644	valid_0's binary_logloss: 0.431703
[400]	valid_0's auc: 0.784746	valid_0's binary_logloss: 0.431784

AUC Score: 0.7844754964641012  # was ignored.

:::::::::::::::::: Fold ~ 3 :::::::::::::::::::
ExtraTreesClassfier >>
LGBMClassifier >>



...

[100]	valid_0's auc: 0.79776	valid_0's binary_logloss: 0.428513
[200]	valid_0's auc: 0.797616	valid_0's binary_logloss: 0.428597
[300]	valid_0's auc: 0.796944	valid_0's binary_logloss: 0.429389

AUC Score: 0.797761354801809  # it is ok.

:::::::::::::::::: Fold ~ 45 :::::::::::::::::::
HGBClassifier >>

AUC Score: 0.7912844103885619  # it is ok.

:::::::::::::::::: Fold ~ 46 :::::::::::::::::::
HGBClassifier >>

AUC Score: 0.7891865376817725  # was ignored.

:::::::::::::::::: Fold ~ 47 :::::::::::::::::::
LGBMClassifier >>



/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's auc: 0.789552	valid_0's binary_logloss: 0.427897
[200]	valid_0's auc: 0.789846	valid_0's binary_logloss: 0.427846
[300]	valid_0's auc: 0.789458	valid_0's binary_logloss: 0.42832

AUC Score: 0.7899283617505478  # was ignored.

:::::::::::::::::: Fold ~ 48 :::::::::::::::::::
HGBClassifier >>

AUC Score: 0.7929884028345846  # it is ok.

:::::::::::::::::: Fold ~ 49 :::::::::::::::::::
HGBClassifier >>

AUC Score: 0.7909339464561916  # it is ok.

:::::::::::::::::: Fold ~ 50 :::::::::::::::::::
LGBMClassifier >>



/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's auc: 0.794515	valid_0's binary_logloss: 0.428843
[200]	valid_0's auc: 0.794232	valid_0's binary_logloss: 0.429041
[300]	valid_0's auc: 0.793856	valid_0's binary_logloss: 0.429305

AUC Score: 0.7946002121657749  # it is ok.

 . . . . . . . . . . . . . . . . . . . .
 AUC Score (mean): 0.7938443432119233
 . . . . . . . . . . . . . . . . . . . .

Total number of folds: 50
Number of valid folds: 35
